In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
import scipy
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
path="D:\\CDS513\\Assignment 1\\Data"
    
os.chdir(path)

# Read csv data file
# Data without feature standardization
df = pd.read_csv('Data_final.csv')

In [4]:
df

,User-id,Game-title,Genre,personal-rating
0,5250,Alien Swarm,Shooter,3
1,975449,Alien Swarm,Shooter,5
2,1950243,Alien Swarm,Shooter,3
3,2259650,Alien Swarm,Shooter,2
4,2753525,Alien Swarm,Shooter,2
...,...,...,...,...
42534,287824954,Close Your Eyes,Puzzle,5
42535,293303005,All Is Dust,Role-playing (RPG),5
42536,300248088,Sigils of Elohim,Puzzle,5
42537,306839937,Secret Files Tunguska,Point-and-click,5


# Finding # of interaction

In [5]:
users_interactions_count_df = df.groupby(['User-id', 'Game-title']).size().groupby('User-id').size()
print('# users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5]\
                                    .reset_index()[['User-id']]


users_with_few_interactions_df = users_interactions_count_df[users_interactions_count_df < 3]\
                                    .reset_index()[['User-id']]

print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))
print('# users with less than 3 interactions: %d' % len(users_with_few_interactions_df))

# users: 8476
# users with at least 5 interactions: 1719
# users with less than 3 interactions: 5936


In [6]:
interactions_from_selected_users_df = df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

interactions_from_few_selected_users_df = df.merge(users_with_few_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

print('# of interactions from all users: %d' % len(df))
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))
print('# of interactions from users with less than 3 interactions: %d' % len(interactions_from_few_selected_users_df))

# of interactions from all users: 42539
# of interactions from users with at least 5 interactions: 32809
# of interactions from users with less than 3 interactions: 6932


# Split Dataset for training and testing

In [7]:
interactions_train_df, interactions_test_df = train_test_split(interactions_from_selected_users_df,
                                   stratify=interactions_from_selected_users_df['User-id'],
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 26247
# interactions on Test set: 6562


In [8]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_from_selected_users_df.set_index('User-id')
interactions_train_indexed_df = interactions_train_df.set_index('User-id')
interactions_test_indexed_df = interactions_test_df.set_index('User-id')

In [9]:
def get_items_interacted(UserID, interactions_df):
    # Get the user's data and merge in the game information.
    interacted_items = interactions_df.loc[UserID]['Game-title']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# Top-N accuracy metrics

In [10]:

EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:
    def get_not_interacted_items_sample(self, UserID, sample_size, seed=42):
        interacted_items = get_items_interacted(UserID, interactions_full_indexed_df)
        all_items = set(df['Game-title'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, UserID):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[UserID]
        if type(interacted_values_testset['Game-title']) == pd.Series:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        else:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        interacted_items_count_testset = len(user_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        user_recs_df = model.recommend_items(UserID, 
                                               items_to_ignore=get_items_interacted(UserID, 
                                                                                    interactions_train_indexed_df), 
                                                                                    topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in user_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(UserID, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS)

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = user_recs_df[user_recs_df['Game-title'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['Game-title'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        user_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return user_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, UserID in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            user_metrics = self.evaluate_model_for_user(model, UserID)  
            user_metrics['UserID'] = UserID
            people_metrics.append(user_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['User-id', 'Game-title'])['personal-rating'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 32809


,User-id,Game-title,personal-rating
0,76767,Age of Empires II HD Edition,1.584963
1,76767,Call of Duty Black Ops,2.584963
2,76767,Call of Duty Modern Warfare 2,2.584963
3,76767,Call of Duty Modern Warfare 3,2.000000
4,76767,Call of Duty World at War,2.584963
5,76767,Counter-Strike,2.584963
6,76767,Counter-Strike Global Offensive,1.000000
7,76767,Counter-Strike Source,1.000000
8,76767,Portal 2,2.321928
9,76767,Rise of Nations Extended Edition,2.000000


In [12]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_full_df.pivot(index='User-id', 
                                                          columns='Game-title', 
                                                          values='personal-rating').fillna(0)

users_items_pivot_matrix_df

Game-title,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,140,16bit Trader,18 Wheels of Steel American Long Haul,...,eden*,fault milestone one,iBomber Defense,iBomber Defense Pacific,ibb & obb,rFactor,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,theHunter,theHunter Primal
User-id,,,,,,,,,,,,,,,,,,,,,
76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
554278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285869008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286926470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288274882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[76767,
 86540,
 229911,
 298950,
 554278,
 561758,
 748719,
 975449,
 1129452,
 1612666]

In [15]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

<1719x2530 sparse matrix of type '<class 'numpy.float64'>'
	with 32809 stored elements in Compressed Sparse Row format>

In [16]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [17]:
U.shape

(1719, 15)

In [18]:
Vt.shape

(15, 2530)

In [19]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [20]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 0.03858012, -0.00157282,  0.00888828, ..., -0.02369775,
         0.01491801,  0.00540449],
       [-0.00501989, -0.00020115,  0.00454168, ..., -0.00937024,
        -0.03611946,  0.00205729],
       [ 0.02483409, -0.00212583,  0.00852237, ..., -0.01912498,
         0.01550258,  0.00475537],
       ...,
       [ 0.00514852,  0.00098013, -0.00016914, ..., -0.01114228,
         0.05957575,  0.00425896],
       [ 0.01998306,  0.00114985, -0.00358747, ...,  0.05194149,
         0.03061235,  0.00694315],
       [ 0.00178423,  0.00373751, -0.01109123, ..., -0.01179076,
         0.09533024,  0.01012573]])

In [21]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [22]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,76767,86540,229911,298950,554278,561758,748719,975449,1129452,1612666,...,281535206,282464648,283872900,285386664,285860790,285869008,286926470,288274882,293042025,294620986
Game-title,,,,,,,,,,,,,,,,,,,,,
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),0.366417,0.361385,0.364830,0.359798,0.362412,0.373977,0.356826,0.377256,0.360962,0.360291,...,0.360078,0.362394,0.359092,0.362717,0.359932,0.360589,0.362211,0.362558,0.364271,0.362170
10 Second Ninja,0.361783,0.361941,0.361719,0.362582,0.362017,0.361791,0.362151,0.361753,0.362516,0.361752,...,0.362613,0.362032,0.361766,0.361847,0.361681,0.362033,0.361955,0.362077,0.362097,0.362396
"10,000,000",0.362990,0.362488,0.362948,0.365078,0.362457,0.365294,0.361045,0.367792,0.362645,0.362827,...,0.362031,0.362364,0.361909,0.362035,0.363130,0.361675,0.361861,0.361945,0.361550,0.360684
100% Orange Juice,0.360689,0.362700,0.360795,0.362018,0.361972,0.359977,0.362179,0.360569,0.361534,0.362540,...,0.362528,0.362144,0.361970,0.361968,0.361481,0.362091,0.361791,0.362152,0.362896,0.362938
12 Labours of Hercules,0.363545,0.363608,0.360658,0.353990,0.361619,0.362529,0.364338,0.368978,0.361600,0.364705,...,0.362363,0.361689,0.367103,0.362113,0.367995,0.363407,0.363393,0.363494,0.362620,0.360809
12 Labours of Hercules II The Cretan Bull,0.361864,0.363621,0.358437,0.359274,0.361487,0.362687,0.362161,0.367913,0.360165,0.364511,...,0.360212,0.362034,0.362532,0.362063,0.365534,0.362582,0.363480,0.363049,0.360373,0.361090
12 Labours of Hercules III Girl Power,0.362327,0.363984,0.360869,0.350336,0.361311,0.363879,0.362551,0.367074,0.362548,0.361616,...,0.361419,0.361844,0.364341,0.361779,0.365132,0.362678,0.363306,0.363350,0.361350,0.361254
140,0.362026,0.361812,0.362008,0.361991,0.361895,0.361902,0.361862,0.362319,0.361955,0.362063,...,0.361960,0.361963,0.362107,0.362141,0.361981,0.361941,0.361983,0.362006,0.362103,0.361983
16bit Trader,0.360447,0.362204,0.358541,0.355996,0.363199,0.361588,0.362526,0.371521,0.361818,0.364102,...,0.360785,0.361294,0.362973,0.361569,0.366566,0.362695,0.363398,0.361881,0.360499,0.360450


In [23]:
len(cf_preds_df.columns)

1719

In [24]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['Game-title'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'Game-title', 
                                                          right_on = 'Game-title')[['recStrength', \
                                                                                    'Game-title', \
                                                                                    'Genre']]

        #return self.items_df
        return recommendations_df.drop_duplicates()
    
cf_recommender_model = CFRecommender(cf_preds_df, df)

In [25]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

...

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

1718 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.3508, 'recall@10': 0.4096}


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_16968\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,UserID
51,62,62,62,1.000000,1.000000,62990992
145,49,49,49,1.000000,1.000000,138941587
31,28,35,36,0.777778,0.972222,48798067
302,26,31,36,0.722222,0.861111,11403772
132,27,33,35,0.771429,0.942857,49893565
30,15,34,35,0.428571,0.971429,24469287
440,14,17,32,0.437500,0.531250,116876958
304,15,20,30,0.500000,0.666667,47457723
63,23,26,29,0.793103,0.896552,11373749
56,22,26,28,0.785714,0.928571,10599862


In [26]:
def inspect_interactions(user_id, test_set=True):
    if test_set:
        interactions_df = interactions_test_indexed_df
    else:
        interactions_df = interactions_train_indexed_df
    return interactions_df.loc[user_id]

In [27]:
inspect_interactions(138941587, test_set=False).head(20)

,Game-title,Genre,personal-rating
User-id,,,
138941587,Spelunky,Platform,2
138941587,Lilly and Sasha Curse of the Immortals,Role-playing (RPG),3
138941587,Nux,Shooter,3
138941587,9 Clues The Secret of Serpent Creek,Point-and-click,5
138941587,Age of Empires II HD Edition,Real Time Strategy (RTS),2
138941587,SUPER DISTRO,Adventure,3
138941587,Bad Rats,Platform,3
138941587,Blaster Shooter GunGuy!,Adventure,5
138941587,Mitos.is The Game,Real Time Strategy (RTS),4


In [28]:
cf_recommender_model.recommend_items(138941587, topn=20, verbose=True)

,recStrength,Game-title,Genre
0,0.890370,The Elder Scrolls V Skyrim,Role-playing (RPG)
614,0.785472,PAYDAY The Heist,Shooter
742,0.767098,Super Meat Boy,Platform
797,0.752828,The Binding of Isaac,Shooter
916,0.720718,Borderlands 2,Shooter
1259,0.716306,Half-Life 2,Shooter
1514,0.708454,Trine,Platform
1557,0.704654,Trine 2,Platform
1642,0.697376,AdVenture Capitalist,Simulator
1731,0.696828,Surgeon Simulator,Puzzle


# Nearest Neighbour Model

In [29]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['User-id', 'Game-title'])['personal-rating'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 32809


,User-id,Game-title,personal-rating
0,76767,Age of Empires II HD Edition,1.584963
1,76767,Call of Duty Black Ops,2.584963
2,76767,Call of Duty Modern Warfare 2,2.584963
3,76767,Call of Duty Modern Warfare 3,2.000000
4,76767,Call of Duty World at War,2.584963
5,76767,Counter-Strike,2.584963
6,76767,Counter-Strike Global Offensive,1.000000
7,76767,Counter-Strike Source,1.000000
8,76767,Portal 2,2.321928
9,76767,Rise of Nations Extended Edition,2.000000


## Reshaping Data

In [39]:
wide_data = interactions_full_df.pivot(index = 'User-id', columns = 'Game-title', values = 'personal-rating').fillna(0)
wide_data_sparse = csr_matrix(wide_data.values)
wide_data_sparse

<1719x2530 sparse matrix of type '<class 'numpy.float64'>'
	with 32809 stored elements in Compressed Sparse Row format>

## Fitting the model

In [40]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_data_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

## Making Recommendations

In [62]:
query_index = np.random.choice(wide_data.shape[0])
print (query_index)

distances, indices = model_knn.kneighbors(wide_data.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(wide_data.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_data.index[indices.flatten()[i]], distances.flatten()[i]))

1583
Recommendations for 206908028:

1: 154687199, with distance of 0.42030443976178444:
2: 62078957, with distance of 0.4298063481752191:
3: 113186568, with distance of 0.518564667713641:
4: 283872900, with distance of 0.5525894520269266:
5: 88921433, with distance of 0.5695160409747357:


## Implementing the User-based and Item-based similarity matrix

In [63]:
#Create NumPy array from the data
user_item_matrix = np.array(wide_data)

In [64]:
user_item_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
user_item_matrix.shape

(1719, 2530)

In [68]:
# Function to compute similarity between two users using cosine similarity
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # If any of the vectors is all zeros
    return dot_product / (norm_vector1 * norm_vector2)

# Function to generate user similarity matrix
def generate_user_similarity_matrix(user_item_matrix, similarity_measure=cosine_similarity):
    num_users = user_item_matrix.shape[0]
    similarity_matrix = np.zeros((num_users, num_users))
    for i in range(num_users):
        for j in range(i + 1, num_users):
            similarity_matrix[i, j] = similarity_measure(user_item_matrix[i], user_item_matrix[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]
    return similarity_matrix

# Function to generate item similarity matrix
def generate_item_similarity_matrix(user_item_matrix, similarity_measure=cosine_similarity):
    num_items = user_item_matrix.shape[1]
    similarity_matrix = np.zeros((num_items, num_items))
    for i in range(num_items):
        for j in range(i + 1, num_items):
            similarity_matrix[i, j] = similarity_measure(user_item_matrix[:, i], user_item_matrix[:, j])
            similarity_matrix[j, i] = similarity_matrix[i, j]
    return similarity_matrix

# Function to predict ratings for a user using user-based collaborative filtering
def predict_ratings_user_based(user_item_matrix, similarity_matrix, user_id):
    num_users, num_items = user_item_matrix.shape
    predicted_ratings = np.zeros(num_items)
    for item_id in range(num_items):
        if user_item_matrix[user_id, item_id] == 0:  # If user hasn't rated the item
            numerator = 0
            denominator = 0
            for other_user_id in range(num_users):
                if user_item_matrix[other_user_id, item_id] != 0:  # If other user has rated the item
                    similarity = similarity_matrix[user_id, other_user_id]
                    numerator += similarity * user_item_matrix[other_user_id, item_id]
                    denominator += abs(similarity)
            if denominator != 0:
                predicted_ratings[item_id] = numerator / denominator
    return predicted_ratings

# Function to predict ratings for a user using item-based collaborative filtering
def predict_ratings_item_based(user_item_matrix, similarity_matrix, user_id):
    num_users, num_items = user_item_matrix.shape
    predicted_ratings = np.zeros(num_items)
    for item_id in range(num_items):
        if user_item_matrix[user_id, item_id] == 0:  # If user hasn't rated the item
            rated_users = np.nonzero(user_item_matrix[:, item_id])[0]  # Users who rated this item
            numerator = 0
            denominator = 0
            for other_user_id in rated_users:
                similarity = similarity_matrix[item_id, other_user_id]
                numerator += similarity * user_item_matrix[other_user_id, item_id]
                denominator += abs(similarity)
            if denominator != 0:
                predicted_ratings[item_id] = numerator / denominator
    return predicted_ratings

# Function to calculate Mean Absolute Error (MAE)
def calculate_mae(actual_ratings, predicted_ratings):
    mask = actual_ratings != 0
    return np.mean(np.abs(actual_ratings[mask] - predicted_ratings[mask]))

# Function to calculate Root Mean Squared Error (RMSE)
def calculate_rmse(actual_ratings, predicted_ratings):
    mask = actual_ratings != 0
    return np.sqrt(np.mean((actual_ratings[mask] - predicted_ratings[mask]) ** 2))

# Function to split data into train and test sets
def train_test_split(user_item_matrix, test_size=0.2):
    num_users, num_items = user_item_matrix.shape
    train_matrix = np.copy(user_item_matrix)
    test_matrix = np.zeros((num_users, num_items))
    for user_id in range(num_users):
        rated_items = np.nonzero(user_item_matrix[user_id])[0]
        np.random.shuffle(rated_items)
        test_items = rated_items[:int(len(rated_items) * test_size)]
        train_matrix[user_id, test_items] = 0
        test_matrix[user_id, test_items] = user_item_matrix[user_id, test_items]
    return train_matrix, test_matrix

# Function to evaluate the recommender system
def evaluate_recommender_system(user_item_matrix, similarity_matrix_user, similarity_matrix_item, test_matrix):
    num_users = user_item_matrix.shape[0]
    total_mae_user_based = 0
    total_rmse_user_based = 0
    total_mae_item_based = 0
    total_rmse_item_based = 0
    num_evaluated_users = 0
    for user_id in range(num_users):
        if np.any(test_matrix[user_id]):  # User has rated items in the test set
            # User-based collaborative filtering
            predicted_ratings_user_based = predict_ratings_user_based(user_item_matrix, similarity_matrix_user, user_id)
            actual_ratings = test_matrix[user_id]
            print("User:", user_id)
            print("Actual Interactions:", actual_ratings)
            print("Predicted Interactions User-Based:", predicted_ratings_user_based)
            mae_user_based = calculate_mae(actual_ratings, predicted_ratings_user_based)
            rmse_user_based = calculate_rmse(actual_ratings, predicted_ratings_user_based)
            print("MAE User-Based:", mae_user_based)
            print("RMSE User-Based:", rmse_user_based)
            total_mae_user_based += mae_user_based
            total_rmse_user_based += rmse_user_based

            # Item-based collaborative filtering
            predicted_ratings_item_based = predict_ratings_item_based(user_item_matrix, similarity_matrix_item, user_id)
            print("Predicted Interactions Item-Based:", predicted_ratings_item_based)
            mae_item_based = calculate_mae(actual_ratings, predicted_ratings_item_based)
            rmse_item_based = calculate_rmse(actual_ratings, predicted_ratings_item_based)
            print("MAE Item-Based:", mae_item_based)
            print("RMSE Item-Based:", rmse_item_based)
            total_mae_item_based += mae_item_based
            total_rmse_item_based += rmse_item_based

            num_evaluated_users += 1

    if num_evaluated_users == 0:
        return None, None, None, None

    avg_mae_user_based = total_mae_user_based / num_evaluated_users
    avg_rmse_user_based = total_rmse_user_based / num_evaluated_users
    avg_mae_item_based = total_mae_item_based / num_evaluated_users
    avg_rmse_item_based = total_rmse_item_based / num_evaluated_users

    return avg_mae_user_based, avg_rmse_user_based, avg_mae_item_based, avg_rmse_item_based

# Apply the functions
train_matrix, test_matrix = train_test_split(user_item_matrix, test_size=0.2)
similarity_matrix_user = generate_user_similarity_matrix(train_matrix)
similarity_matrix_item = generate_item_similarity_matrix(train_matrix)
avg_mae_user_based, avg_rmse_user_based, avg_mae_item_based, avg_rmse_item_based = evaluate_recommender_system(train_matrix, similarity_matrix_user, similarity_matrix_item, test_matrix)

print("User-based collaborative filtering:")
print("Average MAE:", avg_mae_user_based)
print("Average RMSE:", avg_rmse_user_based)
print("\nItem-based collaborative filtering:")
print("Average MAE:", avg_mae_item_based)
print("Average RMSE:", avg_rmse_item_based)

User: 0
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.05880628 2.5849625  2.5849625  ... 2.20259536 2.10822179 1.69703307]
MAE User-Based: 0.3452002228171662
RMSE User-Based: 0.481963967619106
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.3871991635494335
RMSE Item-Based: 0.5441108216628869
User: 1
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.07297247 2.5849625  2.5849625  ... 2.19501969 1.98374933 1.84151929]
MAE User-Based: 0.8229695503563332
RMSE User-Based: 0.8229695503563332
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.8363721844034193
RMSE Item-Based: 0.8363721844034193
User: 2
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.5849625  0.         0.         ... 2.11860564 1.5849625  0.        ]
MAE User-Based: 0.4271772225306032


User: 21
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.11227358 2.5849625  2.5849625  ... 2.12615332 1.82675762 1.66380339]
MAE User-Based: 0.8368021778308528
RMSE User-Based: 0.8368021778308528
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.8265247447183706
RMSE Item-Based: 0.8265247447183706
User: 22
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.19983619 2.5849625  2.5849625  ... 2.19096021 1.8407477  1.71898607]
MAE User-Based: 0.6474587121508288
RMSE User-Based: 0.6553619649657345
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.7726804268066483
RMSE Item-Based: 0.7750129215239294
User: 23
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.84759467 2.5849625  2.5849625  ... 2.17393872 2.02469042 1.71841022]
MAE User-Based: 1.3686483962478

User: 42
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.01477401 2.5849625  2.5849625  ... 2.18175718 1.85222112 0.        ]
MAE User-Based: 0.7368921259891309
RMSE User-Based: 0.7387900879636945
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.7863959583521937
RMSE Item-Based: 0.7877617410253637
User: 43
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.86261625 2.5849625  2.5849625  ... 2.35842684 1.76607089 0.        ]
MAE User-Based: 0.19373402828903252
RMSE User-Based: 0.19373402828903252
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 2.584962500721156
RMSE Item-Based: 2.584962500721156
User: 44
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.07028693 2.5849625  2.5849625  ... 2.24788688 2.04565392 1.85229545]
MAE User-Based: 0.4712126902807

User: 63
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.10224741 2.5849625  2.5849625  ... 2.12119271 1.92830754 1.80013723]
MAE User-Based: 0.7293546243785575
RMSE User-Based: 1.021311481253234
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.6267737463666645
RMSE Item-Based: 0.9890576377077135
User: 64
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.04306699 2.5849625  2.5849625  ... 2.19518903 1.83249131 2.12236704]
MAE User-Based: 0.6477985416269129
RMSE User-Based: 0.7327164370887186
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.6836826721001263
RMSE Item-Based: 0.7961058293585493
User: 65
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.15229987 2.5849625  2.5849625  ... 2.16321116 1.96539156 1.5849625 ]
MAE User-Based: 0.38416983825336

User: 84
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.99988618 2.5849625  0.         ... 2.16051897 1.92005841 2.26617732]
MAE User-Based: 0.9130210581948943
RMSE User-Based: 1.144027952945848
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 1.076590026533634
RMSE Item-Based: 1.2753238725326361
User: 85
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.0325779  2.5849625  2.5849625  ... 2.15349379 2.00699301 2.5849625 ]
MAE User-Based: 0.44744717883660345
RMSE User-Based: 0.44744717883660345
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.44378447646480024
RMSE Item-Based: 0.44378447646480024
User: 86
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.77789759 2.5849625  2.5849625  ... 2.12883957 2.02416679 0.        ]
MAE User-Based: 0.41661479177

...

User: 1617
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.13790076 2.5849625  2.5849625  ... 2.16371872 1.91442626 1.10945722]
MAE User-Based: 1.040084397489175
RMSE User-Based: 1.040084397489175
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 2.584962500721156
RMSE Item-Based: 2.584962500721156
User: 1618
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.5849625  2.5849625  0.         ... 2.05684205 0.         1.2355332 ]
MAE User-Based: 0.29687560430394133
RMSE User-Based: 0.3118138982680697
Predicted Interactions Item-Based: [0. 0. 0. ... 2. 0. 0.]
MAE Item-Based: 0.3346210106356388
RMSE Item-Based: 0.3546566276192831
User: 1619
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.92338721 2.5849625  2.5849625  ... 2.1475992  1.99978396 2.5849625 ]
MAE User-Based: 1.0038841527939764
RMSE User-Based: 1.0038841527939764

User: 1638
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.5849625  0.         2.5849625  ... 2.5849625  2.32192809 1.5849625 ]
MAE User-Based: 0.7240606149660309
RMSE User-Based: 0.7240606149660309
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.6087458521409714
RMSE Item-Based: 0.6087458521409714
User: 1639
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.04590426 2.5849625  2.5849625  ... 2.07399388 1.99134065 1.89502806]
MAE User-Based: 0.40733692597532234
RMSE User-Based: 0.40733692597532234
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 1.5849625007211563
RMSE Item-Based: 1.5849625007211563
User: 1640
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.13020316 2.5849625  2.5849625  ... 2.0510537  1.81542865 1.5849625 ]
MAE User-Based: 0.86125

User: 1659
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.6628494  0.         2.5849625  ... 2.1472804  1.93221898 2.04103649]
MAE User-Based: 0.42420468398085653
RMSE User-Based: 0.42639349584182407
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 1.549130171866886
RMSE Item-Based: 1.863532372374569
User: 1660
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.10099005 2.5849625  2.5849625  ... 2.114001   1.94002403 1.62869046]
MAE User-Based: 0.07026098106249323
RMSE User-Based: 0.07026098106249323
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.28384851814957446
RMSE Item-Based: 0.28384851814957446
User: 1661
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.75426425 2.5849625  0.         ... 2.08440644 2.10711992 1.92774666]
MAE User-Based: 0.627

User: 1680
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.84511137 0.         2.5849625  ... 2.26675149 1.86854419 2.5849625 ]
MAE User-Based: 0.5773510340231045
RMSE User-Based: 0.5773510340231045
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.7568709985853725
RMSE Item-Based: 0.7568709985853725
User: 1681
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [1.5849625  2.5849625  0.         ... 1.88398189 1.5849625  1.34552234]
MAE User-Based: 0.6248845816281667
RMSE User-Based: 0.6248845816281667
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.7841074333440954
RMSE Item-Based: 0.7841074333440954
User: 1682
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.35244079 2.5849625  0.         ... 2.06962872 2.01375511 1.85573846]
MAE User-Based: 0.7601408

User: 1701
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.09753138 2.5849625  2.5849625  ... 2.14713034 2.06816958 1.95057849]
MAE User-Based: 0.15969376337627894
RMSE User-Based: 0.15969376337627894
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.11417672126127076
RMSE Item-Based: 0.11417672126127076
User: 1702
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.5849625 0.        2.5849625 ... 2.        0.        0.       ]
MAE User-Based: 0.3175477252406589
RMSE User-Based: 0.3175477252406589
Predicted Interactions Item-Based: [0.        0.        0.        ... 2.        1.5849625 0.       ]
MAE Item-Based: 0.4493305603248101
RMSE Item-Based: 0.4493305603248101
User: 1703
Actual Interactions: [0. 0. 0. ... 0. 0. 0.]
Predicted Interactions User-Based: [2.11500994 2.5849625  2.5849625  ... 2.24651999 1.83335653 1.        ]
MAE User-Based: 1.092557312

In [84]:
def display_recommendations(predicted_ratings):
    for idx, rating in enumerate(predicted_ratings):
        if rating > 0:  # Only display items with positive predicted ratings
            print(f"Item {idx} with predicted rating {rating}")

# Predict ratings for a specific user
user_id = 0
predicted_ratings_user_based = predict_ratings_user_based(train_matrix, similarity_matrix_user, user_id)
predicted_ratings_item_based = predict_ratings_item_based(train_matrix, similarity_matrix_item, user_id)

In [82]:
# Display the Item-based recommendations
print(f"\nItem-based recommendation for user {user_id}:")
display_recommendations(predicted_ratings_item_based)


Item-based recommendation for user 0:
Item 13 with predicted rating 1.0
Item 16 with predicted rating 1.6809342924222044
Item 34 with predicted rating 1.7336504348380222
Item 35 with predicted rating 1.5531675397808216
Item 44 with predicted rating 1.7849440608815204
Item 46 with predicted rating 1.7862536308208252
Item 51 with predicted rating 1.0
Item 54 with predicted rating 1.8877177681402857
Item 56 with predicted rating 1.5800373148334006
Item 57 with predicted rating 1.7708699786350957
Item 58 with predicted rating 1.0
Item 59 with predicted rating 2.1103811835624273
Item 62 with predicted rating 2.247755240416299
Item 69 with predicted rating 2.0594116479057196
Item 74 with predicted rating 1.9607257848375885
Item 78 with predicted rating 1.9410626420606663
Item 82 with predicted rating 1.3549265815759914
Item 84 with predicted rating 2.1502639910027455
Item 85 with predicted rating 2.584962500721156
Item 87 with predicted rating 1.7911916500954628
Item 91 with predicted ratin

In [83]:
# Display the User-based recommendations 
print(f"User-based recommendation for user {user_id}:")
display_recommendations(predicted_ratings_user_based)

User-based recommendation for user 0:
Item 0 with predicted rating 2.0588062762174784
Item 1 with predicted rating 2.584962500721156
Item 2 with predicted rating 2.584962500721156
Item 3 with predicted rating 1.4537823800857417
Item 4 with predicted rating 2.534913712457303
Item 5 with predicted rating 2.2601767308450578
Item 6 with predicted rating 2.421583826465426
Item 8 with predicted rating 2.584962500721156
Item 10 with predicted rating 2.584962500721156
Item 11 with predicted rating 2.584962500721156
Item 13 with predicted rating 1.5498169887754065
Item 14 with predicted rating 2.584962500721156
Item 16 with predicted rating 1.3521807885280115
Item 18 with predicted rating 2.584962500721156
Item 19 with predicted rating 2.584962500721156
Item 20 with predicted rating 2.5849625007211565
Item 22 with predicted rating 2.584962500721156
Item 23 with predicted rating 2.584962500721156
Item 24 with predicted rating 2.070085348905754
Item 25 with predicted rating 1.5849625007211563
Ite

In [ ]:
similarity_matrix_user

In [ ]:
inspect_interactions(138941587, test_set=False).head(20)

In [ ]:
user_item_matrix

In [ ]:
# Define the CFRecommender class as before
class MemoryCFRecommender:
    
    MODEL_NAME = 'Memory Based Collaborative Filtering'
    
    def __init__(self, user_item_matrix, similarity_matrix_user, similarity_matrix_item, items_df=None):
        self.user_item_matrix = user_item_matrix
        self.similarity_matrix_user = similarity_matrix_user
        self.similarity_matrix_item = similarity_matrix_item
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def predict_ratings_user_based(self, user_id):
        num_users, num_items = self.user_item_matrix.shape
        predicted_ratings = np.zeros(num_items)
        for item_id in range(num_items):
            if self.user_item_matrix[user_id, item_id] == 0:  # If user hasn't rated the item
                numerator = 0
                denominator = 0
                for other_user_id in range(num_users):
                    if self.user_item_matrix[other_user_id, item_id] != 0:  # If other user has rated the item
                        similarity = self.similarity_matrix_user[user_id, other_user_id]
                        numerator += similarity * self.user_item_matrix[other_user_id, item_id]
                        denominator += abs(similarity)
                if denominator != 0:
                    predicted_ratings[item_id] = numerator / denominator
        return predicted_ratings
    
    def predict_ratings_item_based(self, user_id):
        num_users, num_items = self.user_item_matrix.shape
        predicted_ratings = np.zeros(num_items)
        for item_id in range(num_items):
            if self.user_item_matrix[user_id, item_id] == 0:  # If user hasn't rated the item
                rated_users = np.nonzero(self.user_item_matrix[:, item_id])[0]  # Users who rated this item
                numerator = 0
                denominator = 0
                for other_user_id in rated_users:
                    similarity = self.similarity_matrix_item[item_id, other_user_id]
                    numerator += similarity * self.user_item_matrix[other_user_id, item_id]
                    denominator += abs(similarity)
                if denominator != 0:
                    predicted_ratings[item_id] = numerator / denominator
        return predicted_ratings
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, use_item_based=False, verbose=False):
        if use_item_based:
            predicted_ratings = self.predict_ratings_item_based(user_id)
        else:
            predicted_ratings = self.predict_ratings_user_based(user_id)
        
        # Create a DataFrame of predicted ratings
        predictions_df = pd.DataFrame({'Game-title': np.arange(len(predicted_ratings)),\
                                       'recStrength': predicted_ratings})
        
        # Filter out items to ignore
        recommendations_df = predictions_df[~predictions_df['Game-title'].isin(items_to_ignore)] \
                                .sort_values('recStrength', ascending=False) \
                                .head(topn)
        
        if verbose and self.items_df is not None:
            recommendations_df = recommendations_df.merge(self.items_df, how='left', on='Game-title') \
                                                   [['recStrength', 'Game-title', 'Genre']]
        elif verbose and self.items_df is None:
            raise Exception('"items_df" is required in verbose mode')
        
        return recommendations_df.drop_duplicates()

# Create an instance of CFRecommender
cf_recommender_model = MemoryCFRecommender(user_item_matrix, similarity_matrix_user, similarity_matrix_item, items_df)

# Example usage
user_id = 0  # Change this to the user ID you want to get recommendations for
items_to_ignore = []  # List of items to ignore
topn = 5  # Number of top recommendations to return
use_item_based = False  # Change to True for item-based recommendations
verbose = True  # Set to True to include item details in the recommendations

In [ ]:
# Get recommendations
recommendations = cf_recommender_model.recommend_items(user_id, \
                                                       topn, \
                                                       use_item_based, \
                                                       verbose)
print("Recommendations for User", user_id)
print(recommendations)